# Exploring the Raw Data Structure
In this notebook, we investigate the provided row counts matrices, select the existing matrices and merge them using scanpy and anndata.

In [2]:
! ls -lh /media/gambino/students_workdir/ibp/data 

total 96K
drwxrwxrwx 3 cey cey 4,0K Nov  6 11:55 GC1004281_E2
drwxrwxrwx 3 cey cey 4,0K Nov  6 11:55 GC1004282_F2
drwxrwxrwx 3 cey cey 4,0K Nov  6 11:55 GC1004283_G2
drwxrwxrwx 3 cey cey 4,0K Nov  6 11:56 GC1004284_F4
drwxrwxrwx 3 cey cey 4,0K Nov  6 11:56 GC1004285_G4
drwxrwxrwx 3 cey cey 4,0K Nov  6 11:57 GC1004286_H4
drwxrwxrwx 3 cey cey 4,0K Nov  6 11:57 GC108651_SI-GA-C3
drwxrwxrwx 3 cey cey 4,0K Nov  6 11:57 GC108652_SI-GA-D3
drwxrwxrwx 3 cey cey 4,0K Nov  6 11:57 GC108653_SI-GA-E3
drwxrwxrwx 3 cey cey 4,0K Nov  6 11:58 GC110299_SI-GA-E8
drwxrwxrwx 3 cey cey 4,0K Nov  6 11:58 GC110300_SI-GA-F8
drwxrwxrwx 3 cey cey 4,0K Nov  6 11:59 GC110301_SI-GA-G8
drwxrwxrwx 3 cey cey 4,0K Nov  6 11:59 GC110994_SI-GA-B6
drwxrwxrwx 3 cey cey 4,0K Nov  6 11:59 GC110995_SI-GA-C6
drwxrwxrwx 3 cey cey 4,0K Nov  8 13:45 GC110996_SI-GA-D6
drwxrwxrwx 3 cey cey 4,0K Nov  8 13:45 GC111283_SI-GA-D7
drwxrwxrwx 3 cey cey 4,0K Nov  8 13:46 GC111284_SI-GA-E7
drwxrwxrwx 3 cey cey 4,0K Nov  8 13:48 GC111285_SI-

In [24]:
import os
import pandas as pd

DATA_DIR = "/media/gambino/students_workdir/ibp/data"
samples = sorted(os.listdir(DATA_DIR))
samples_desc = pd.read_excel("/media/gambino/students_workdir/ibp/Colon_ERC.xlsx")

missing_samples = samples_desc.loc[~samples_desc["GC code"].isin(samples), "GC code"]
print("Number of Existing Sample Folders:", len(samples))
existing_raw_matrices=[]
print("##################################################################################################")
for sample in samples:
    sample_path = os.path.join(DATA_DIR, sample)
    contents = os.listdir(os.path.join(sample_path, "outs"))
    tissue_row = samples_desc.loc[samples_desc["GC code"] == sample, "layer"]

    if not tissue_row.empty:
        tissue = tissue_row.iloc[0]
    else:
        tissue = "N/A"

    has_filtered_matrix = "filtered_feature_bc_matrix.h5" in contents
    has_raw_matrix = "raw_feature_bc_matrix.h5" in contents
    if has_raw_matrix:
        existing_raw_matrices.append(sample)
    # Calculate folder size
    folder_size = sum(os.path.getsize(os.path.join(sample_path, "outs",f)) for f in contents)

    # Prettify the output
    print(
        f"{sample:<18} - Tissue: {tissue:<15}, "
        f"filtered_feature_bc_matrix.h5: {has_filtered_matrix:<2}, "
        f"raw_feature_bc_matrix.h5: {has_raw_matrix:<2}, "
        f"Folder Size: {folder_size / (1024 * 1024):.2f} MB"
    )
print("##################################################################################################")
print("Missing Samples Folders:", missing_samples.values.tolist())
print(f"Existing Raw Files:{existing_raw_matrices}")

Number of Existing Sample Folders: 24
##################################################################################################
GC1004281_E2       - Tissue: Muscularis     , filtered_feature_bc_matrix.h5: 1 , raw_feature_bc_matrix.h5: 1 , Folder Size: 403.31 MB
GC1004282_F2       - Tissue: Submucosa      , filtered_feature_bc_matrix.h5: 1 , raw_feature_bc_matrix.h5: 1 , Folder Size: 437.70 MB
GC1004283_G2       - Tissue: Lamina Propria , filtered_feature_bc_matrix.h5: 1 , raw_feature_bc_matrix.h5: 1 , Folder Size: 285.79 MB
GC1004284_F4       - Tissue: Muscularis     , filtered_feature_bc_matrix.h5: 1 , raw_feature_bc_matrix.h5: 1 , Folder Size: 499.47 MB
GC1004285_G4       - Tissue: Submucosa      , filtered_feature_bc_matrix.h5: 1 , raw_feature_bc_matrix.h5: 1 , Folder Size: 481.75 MB
GC1004286_H4       - Tissue: Lamina Propria , filtered_feature_bc_matrix.h5: 1 , raw_feature_bc_matrix.h5: 1 , Folder Size: 501.58 MB
GC108651_SI-GA-C3  - Tissue: Muscularis     , filtered_feat

Looks like there are quite some missing sample folder all together, and some folders with no raw counts matrices inside. We need to discuss these anomalies with the lab supervisor.
Next, we merge the different files and assign each their corresponding sample ids.

In [54]:
import scanpy as sc

adatas = [] 
for filename in existing_raw_matrices: 
    adata = sc.read_10x_h5(os.path.join(DATA_DIR, filename, "outs", "raw_feature_bc_matrix.h5"))
    adata.var_names_make_unique() 
    adata.obs['batch'] = filename
    adatas.append(adata) 

combined = adatas[0].concatenate(adatas[1:], batch_key='batch', batch_categories=existing_raw_matrices)


/media/gambino/students_workdir/anis/miniconda3/envs/pytorch-env/lib/python3.9/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/media/gambino/students_workdir/anis/miniconda3/envs/pytorch-env/lib/python3.9/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/media/gambino/students_workdir/anis/miniconda3/envs/pytorch-env/lib/python3.9/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/media/gambino/students_workdir/anis/miniconda3/envs/pytorch-env/lib/python3.9/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.w

Quick quality checks:

In [55]:
print(sum(adata.shape[0] for adata in adatas))
print(combined.shape[0])

20266124
20266124


In [56]:
combined.obs_names,combined.obs_keys

(Index(['AAACCCAAGAAACACT-1-GC1004281_E2', 'AAACCCAAGAAACCAT-1-GC1004281_E2',
        'AAACCCAAGAAACCCA-1-GC1004281_E2', 'AAACCCAAGAAACCCG-1-GC1004281_E2',
        'AAACCCAAGAAACTAC-1-GC1004281_E2', 'AAACCCAAGAAACTCA-1-GC1004281_E2',
        'AAACCCAAGAAAGTCT-1-GC1004281_E2', 'AAACCCAAGAAATCCA-1-GC1004281_E2',
        'AAACCCAAGAAATGGG-1-GC1004281_E2', 'AAACCCAAGAAATTGC-1-GC1004281_E2',
        ...
        'TTTGTTGTCTTGGAAC-1-GC110299_SI-GA-E8',
        'TTTGTTGTCTTGGCTC-1-GC110299_SI-GA-E8',
        'TTTGTTGTCTTGTGCC-1-GC110299_SI-GA-E8',
        'TTTGTTGTCTTGTTAC-1-GC110299_SI-GA-E8',
        'TTTGTTGTCTTTCCAA-1-GC110299_SI-GA-E8',
        'TTTGTTGTCTTTCTAG-1-GC110299_SI-GA-E8',
        'TTTGTTGTCTTTGCAT-1-GC110299_SI-GA-E8',
        'TTTGTTGTCTTTGCTA-1-GC110299_SI-GA-E8',
        'TTTGTTGTCTTTGCTG-1-GC110299_SI-GA-E8',
        'TTTGTTGTCTTTGGAG-1-GC110299_SI-GA-E8'],
       dtype='object', length=20266124),
 <bound method AnnData.obs_keys of AnnData object with n_obs × n_vars = 2026

In [67]:
combined.obs["batch"].value_counts()

GC1004286_H4         2519119
GC1004284_F4         2437891
GC1004281_E2         2162705
GC1004285_G4         2150640
GC108653_SI-GA-E3    2114310
GC1004282_F2         1982651
GC110299_SI-GA-E8    1945915
GC108652_SI-GA-D3    1933702
GC1004283_G2         1715506
GC108651_SI-GA-C3    1303685
Name: batch, dtype: int64

Finally, we will save the combined file in the IBP folder:

In [71]:
combined.write(os.path.join("/media/gambino/students_workdir/ibp","combined_raw_counts_12_11.h5"))

In [74]:
import anndata
loaded_adata = anndata.read(os.path.join("/media/gambino/students_workdir/ibp","combined_raw_counts_12_11.h5"))
loaded_adata.obs["batch"].value_counts()

GC1004286_H4         2519119
GC1004284_F4         2437891
GC1004281_E2         2162705
GC1004285_G4         2150640
GC108653_SI-GA-E3    2114310
GC1004282_F2         1982651
GC110299_SI-GA-E8    1945915
GC108652_SI-GA-D3    1933702
GC1004283_G2         1715506
GC108651_SI-GA-C3    1303685
Name: batch, dtype: int64